In [1]:
import tensorflow as tf
import tensorflow_text as text
import numpy as np
import functools

2024-03-18 08:40:05.251429: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-18 08:40:05.354729: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-18 08:40:05.354778: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-18 08:40:05.360225: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-18 08:40:05.385985: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-18 08:40:05.389363: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
examples = {
    "text_a": [
      "Sponge bob Squarepants is an Avenger",
      "Marvel Avengers"
    ],
    "text_b": [
     "Barack Obama is the President.",
     "President is the highest office"
  ],
}

dataset = tf.data.Dataset.from_tensor_slices(examples)
next(iter(dataset))

{'text_a': <tf.Tensor: shape=(), dtype=string, numpy=b'Sponge bob Squarepants is an Avenger'>,
 'text_b': <tf.Tensor: shape=(), dtype=string, numpy=b'Barack Obama is the President.'>}

In [3]:
_VOCAB = [
    # Special tokens
    b"[UNK]", b"[MASK]", b"[RANDOM]", b"[CLS]", b"[SEP]",
    # Suffixes
    b"##ack", b"##ama", b"##ger", b"##gers", b"##onge", b"##pants",  b"##uare",
    b"##vel", b"##ven", b"an", b"A", b"Bar", b"Hates", b"Mar", b"Ob",
    b"Patrick", b"President", b"Sp", b"Sq", b"bob", b"box", b"has", b"highest",
    b"is", b"office", b"the",
]

_START_TOKEN = _VOCAB.index(b"[CLS]")
_END_TOKEN = _VOCAB.index(b"[SEP]")
_MASK_TOKEN = _VOCAB.index(b"[MASK]")
_RANDOM_TOKEN = _VOCAB.index(b"[RANDOM]")
_UNK_TOKEN = _VOCAB.index(b"[UNK]")
_MAX_SEQ_LEN = 8
_MAX_PREDICTIONS_PER_BATCH = 5

_VOCAB_SIZE = len(_VOCAB)

In [4]:
lookup_table = tf.lookup.StaticVocabularyTable(
    tf.lookup.KeyValueTensorInitializer(keys=_VOCAB,
                                        key_dtype=tf.string,
                                        values=tf.range(tf.size(_VOCAB, out_type=tf.int64), dtype=tf.int64),
                                        value_dtype=tf.int64
                                       ),
    num_oov_buckets=1
)

In [18]:
text_a = examples['text_a']
text_b = examples['text_b']
vocab_table=lookup_table
tokenizer = text.BertTokenizer(vocab_table, token_out_type=tf.int64)
segments = [tokenizer.tokenize(text).merge_dims(-2,-1) for text in [text_a, text_b]]
trimmer = text.RoundRobinTrimmer(max_seq_length=_MAX_SEQ_LEN)
trimmed_segments = trimmer.trim(segments)
segments_combined, segments_id = text.combine_segments(trimmed_segments, start_of_sequence_id=_START_TOKEN, end_of_segment_id=_END_TOKEN)
segments_combined, segments_id
random_selector = text.RandomItemSelector(max_selections_per_batch=_MAX_PREDICTIONS_PER_BATCH, selection_rate=0.2,unselectable_ids=[_START_TOKEN, _UNK_TOKEN, _END_TOKEN])
mask_values_chooser = text.MaskValuesChooser(_VOCAB_SIZE, _MASK_TOKEN, mask_token_rate=0.8)
input_word_ids, masked_lm_positions, masked_lm_ids = text.mask_language_model(segments_combined, random_selector, mask_values_chooser)
input_word_ids, masked_lm_positions, masked_lm_ids

(<tf.RaggedTensor [[3, 22, 9, 24, 23, 4, 1, 5, 19, 1, 4],
  [3, 1, 12, 1, 13, 4, 21, 28, 30, 27, 4]]>,
 <tf.RaggedTensor [[6, 9],
  [1, 3]]>,
 <tf.RaggedTensor [[16, 6],
  [18, 15]]>)

In [23]:
def bert_pretrain_preprocess(vocab_table, features):
    text_a = features['text_a']
    text_b = features['text_b']
    tokenizer = text.BertTokenizer(vocab_table, token_out_type=tf.int64)
    segments = [tokenizer.tokenize(text).merge_dims(-2,-1) for text in [text_a, text_b]]
    trimmer = text.RoundRobinTrimmer(max_seq_length=_MAX_SEQ_LEN)
    trimmed_segments = trimmer.trim(segments)
    segments_combined, segments_id = text.combine_segments(trimmed_segments, start_of_sequence_id=_START_TOKEN, end_of_segment_id=_END_TOKEN)
    random_selector = text.RandomItemSelector(max_selections_per_batch=_MAX_PREDICTIONS_PER_BATCH, selection_rate=0.2,unselectable_ids=[_START_TOKEN, _UNK_TOKEN, _END_TOKEN])
    mask_values_chooser = text.MaskValuesChooser(_VOCAB_SIZE, _MASK_TOKEN, mask_token_rate=0.8)
    input_word_ids, masked_lm_positions, masked_lm_ids = text.mask_language_model(segments_combined, random_selector, mask_values_chooser)
    input_word_ids, input_mask = text.pad_model_inputs(input_word_ids, max_seq_length=_MAX_SEQ_LEN)
    input_type_ids, _ = text.pad_model_inputs(segments_id, max_seq_length=_MAX_SEQ_LEN)
    masked_lm_positions, masked_lm_weights = text.pad_model_inputs(masked_lm_positions, max_seq_length=_MAX_PREDICTIONS_PER_BATCH)
    masked_lm_ids, _ = text.pad_model_inputs(masked_lm_ids, max_seq_length=_MAX_PREDICTIONS_PER_BATCH)
    model = {
        "input_word_ids":input_word_ids,
        "input_mask":input_mask,
        "input_type_ids":input_type_ids,
        "masked_lm_positions" : masked_lm_positions,
        "masked_lm_weights": masked_lm_weights,
        "masked_lm_ids":masked_lm_ids
    }
    return model
    

In [24]:
dataset = (
    tf.data.Dataset.from_tensors(examples)
    .map(functools.partial(bert_pretrain_preprocess, lookup_table))
)

next(iter(dataset))

{'input_word_ids': <tf.Tensor: shape=(2, 8), dtype=int64, numpy=
 array([[ 3, 22,  9, 24,  1,  4, 16,  5],
        [ 3, 18, 12, 15, 13,  4,  1,  1]])>,
 'input_mask': <tf.Tensor: shape=(2, 8), dtype=int64, numpy=
 array([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1]])>,
 'input_type_ids': <tf.Tensor: shape=(2, 8), dtype=int64, numpy=
 array([[0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 1]])>,
 'masked_lm_positions': <tf.Tensor: shape=(2, 5), dtype=int64, numpy=
 array([[4, 9, 0, 0, 0],
        [6, 7, 0, 0, 0]])>,
 'masked_lm_weights': <tf.Tensor: shape=(2, 5), dtype=int64, numpy=
 array([[1, 1, 0, 0, 0],
        [1, 1, 0, 0, 0]])>,
 'masked_lm_ids': <tf.Tensor: shape=(2, 5), dtype=int64, numpy=
 array([[23,  6,  0,  0,  0],
        [21, 28,  0,  0,  0]])>}